## Imports

In [106]:
import os
from os import listdir
import sys
import math
import re
import pandas as pd
import shutil
import copy
import time

import cv2
import numpy as np

import imutils
import matplotlib.pyplot as plt

import tkinter as tk

## Globals & Prepare Model

---

Object Detection

---

In [107]:
#Import Object Detection Model Parameters
OBJ_config = "Model/yolov4.cfg"
OBJ_weights = "Model/yolov4.weights"
OBJ_class_names_file = "Model/coco.names"

# read class names from text file
OBJ_class_names = None
with open(OBJ_class_names_file, 'r') as f:
    OBJ_class_names = [line.strip() for line in f.readlines()]

OBJ_threshold = 0.1

---

Object Tracking

---

In [108]:
TRACK_list = cv2.legacy.MultiTracker_create()

---

Pose Estimation

---

In [109]:
#Import Pose Estimation Model Parameters
POSE_proto = "Model/pose_deploy_linevec.prototxt"
POSE_weights = "Model/pose_iter_440000.caffemodel"

POSE_body_parts = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_part_pairs = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

POSE_threshold = 0.1

In [110]:
#Create Models
OBJ_net = cv2.dnn.readNet(OBJ_config, OBJ_weights)
POSE_net = cv2.dnn.readNetFromCaffe(POSE_proto, POSE_weights)

#Set Models to use GPU
OBJ_net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
OBJ_net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
POSE_net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
POSE_net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)


## DataSet

In [111]:
DATASET_VIDEOS = "Dataset/videos/"
DATASET_CLASSES = ["violent", "non-violent"]
DATASET_CAMS = ["/cam1/","/cam2/"]
DATASET_EXP_LOC = "Dataset/violent_keypoints_dataset.csv"
DATASET_EXP_LOC_CONF = "Dataset/violent_keypoints_conf_dataset.csv"
DATASET_EXP_LOC_LOW = "Dataset/violent_keypoints_low_dataset.csv"
DATASET_EXP_LOC_LOW_CONF = "Dataset/violent_keypoints_low_conf_dataset.csv"


DATASET_COL = [
                    "Class",
                    "Nose_X_1", "Nose_Y_1", "Nose_X_2", "Nose_Y_2", "Nose_X_3", "Nose_Y_3",
                    "Neck_X_1", "Neck_Y_1", "Neck_X_2", "Neck_Y_2", "Neck_X_3", "Neck_Y_3",
                    "RShoulder_X_1", "RShoulder_Y_1", "RShoulder_X_2", "RShoulder_Y_2", "RShoulder_X_3", "RShoulder_Y_3",
                    "RElbow_X_1", "RElbow_Y_1", "RElbow_X_2", "RElbow_Y_2", "RElbow_X_3", "RElbow_Y_3",
                    "RWrist_X_1", "RWrist_Y_1", "RWrist_X_2", "RWrist_Y_2", "RWrist_X_3", "RWrist_Y_3",
                    "LShoulder_X_1", "LShoulder_Y_1", "LShoulder_X_2", "LShoulder_Y_2", "LShoulder_X_3", "LShoulder_Y_3",
                    "LElbow_X_1", "LElbow_Y_1", "LElbow_X_2", "LElbow_Y_2", "LElbow_X_3", "LElbow_Y_3",
                    "LWrist_X_1", "LWrist_Y_1", "LWrist_X_2", "LWrist_Y_2", "LWrist_X_3", "LWrist_Y_3",
                    "RHip_X_1", "RHip_Y_1", "RHip_X_2", "RHip_Y_2", "RHip_X_3", "RHip_Y_3",
                    "RKnee_X_1", "RKnee_Y_1", "RKnee_X_2", "RKnee_Y_2", "RKnee_X_3", "RKnee_Y_3",
                    "RAnkle_X_1", "RAnkle_Y_1", "RAnkle_X_2", "RAnkle_Y_2", "RAnkle_X_3", "RAnkle_Y_3",
                    "LHip_X_1", "LHip_Y_1", "LHip_X_2", "LHip_Y_2", "LHip_X_3", "LHip_Y_3",
                    "LKnee_X_1", "LKnee_Y_1", "LKnee_X_2", "LKnee_Y_2", "LKnee_X_3", "LKnee_Y_3",
                    "LAnkle_X_1", "LAnkle_Y_1", "LAnkle_X_2", "LAnkle_Y_2", "LAnkle_X_3", "LAnkle_Y_3",
                    "REye_X_1", "REye_Y_1", "REye_X_2", "REye_Y_2", "REye_X_3", "REye_Y_3",
                    "LEye_X_1", "LEye_Y_1", "LEye_X_2", "LEye_Y_2", "LEye_X_3", "LEye_Y_3",
                    "REar_X_1", "REar_Y_1", "REar_X_2", "REar_Y_2", "REar_X_3", "REar_Y_3",
                    "LEar_X_1", "LEar_Y_1", "LEar_X_2", "LEar_Y_2", "LEar_X_3", "LEar_Y_3",
                    "Background_X_1", "Background_Y_1", "Background_X_2", "Background_Y_2", "Background_X_3", "Background_Y_3"
                    ]

DATASET_COL_CONF = [
                        "Class",
                        "Nose_1", "Nose_2", "Nose_3",
                        "Neck_1", "Neck_2", "Neck_3",
                        "RShoulder_1", "RShoulder_2", "RShoulder_3",
                        "RElbow_1", "RElbow_2", "RElbow_3",
                        "RWrist_1", "RWrist_2", "RWrist_3",
                        "LShoulder_1", "LShoulder_2", "LShoulder_3",
                        "LElbow_1", "LElbow_2", "LElbow_3",
                        "LWrist_1", "LWrist_2", "LWrist_3",
                        "RHip_1", "RHip_2", "RHip_3",
                        "RKnee_1", "RKnee_2", "RKnee_3",
                        "RAnkle_1", "RAnkle_2", "RAnkle_3",
                        "LHip_1", "LHip_2", "LHip_3",
                        "LKnee_1", "LKnee_2", "LKnee_3",
                        "LAnkle_1", "LAnkle_2", "LAnkle_3",
                        "REye_1", "REye_2", "REye_3",
                        "LEye_1", "LEye_2", "LEye_3",
                        "REar_1", "REar_2", "REar_3",
                        "LEar_1", "LEar_2", "LEar_3",
                        "Background_1", "Background_2", "Background_3"


]

## Labelling

In [112]:
global window
global check_list
global check_var_list

## Detect People

In [113]:
def get_output_layers(net):

    layer_names = net.getLayerNames()

    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

def detect_individuals_from_image(OBJ_img):

    OBJ_width = OBJ_img.shape[1]
    OBJ_height = OBJ_img.shape[0]
    OBJ_scale = 0.00392

    OBJ_blob = cv2.dnn.blobFromImage(OBJ_img, OBJ_scale, (416, 416), (0,0,0), True, crop=False)

    OBJ_net.setInput(OBJ_blob)
    outs = OBJ_net.forward(get_output_layers(OBJ_net))

    # initialization
    class_ids = []
    confidences = []
    boxes = []
    nms_threshold = 0.4

    # for each detection from each output layer
    # get the confidence, class id, bounding box params
    # and ignore weak detections (confidence < 0.5)
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > OBJ_threshold:
                center_x = int(detection[0] * OBJ_width)
                center_y = int(detection[1] * OBJ_height)
                w = int(detection[2] * OBJ_width)
                h = int(detection[3] * OBJ_height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    #Apply non-max suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, OBJ_threshold, nms_threshold)

    confident_people_box = []

    for i in indices:
        #Only return if detected object is a Person
        if class_ids[i] == 0:
            confident_people_box.append(boxes[i])

    return confident_people_box

## Track People

In [114]:
def start_tracking_from_boxes(TRACK_BBoxes, TRACK_img):

    #New Empty Track list
    new_TRACK_list = cv2.legacy.MultiTracker_create()

    #Fill new Track list with the new Bounding boxes
    for TRACK_box in TRACK_BBoxes:

        tracker = cv2.legacy.TrackerCSRT_create()
        new_TRACK_list.add(tracker, TRACK_img, TRACK_box)

    global TRACK_list
    TRACK_list = new_TRACK_list

def track_using_trackers(TRACK_img, FINAL_img):

    global TRACK_list

    # grab the updated bounding box coordinates (if any) for each object that is being tracked
    (success, TRACK_boxes) = TRACK_list.update(TRACK_img)

def expand_tracking_box(image, FINAL_img):

    global TRACK_list
    TRACK_boxes = TRACK_list.getObjects()
    TRACK_expanded_boxes = [[] for i in range(len(TRACK_boxes))]

    new_OBJ_boxes = detect_individuals_from_image(image)

    # loop over the bounding boxes and draw them on the frame
    for ibox in range(len(TRACK_boxes)):

        x = int(TRACK_boxes[ibox][0])
        y = int(TRACK_boxes[ibox][1])
        w = int(TRACK_boxes[ibox][2])
        h = int(TRACK_boxes[ibox][3])

        cv2.rectangle(FINAL_img, (x,y), (x+w,y+h), (255,0,0), 2)
        cv2.putText(FINAL_img, "Tracker "+str(ibox), (x+10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    #Take account of tracking boxes
    for TRACK_box_i in range(len(TRACK_boxes)):
        b_box = TRACK_boxes[TRACK_box_i]
        TRACK_expanded_boxes[TRACK_box_i].append((int(b_box[0]),int(b_box[1]),int(b_box[2]),int(b_box[3])))

    #Take account new object detection boxes
    for OBJ_box_i in range(len(new_OBJ_boxes)):
        new_OBJ_Center = get_center_of_box(new_OBJ_boxes[OBJ_box_i])

        distance = sys.maxsize
        trackerIndex = -1

        for TRACK_box_i in range(len(TRACK_boxes)):
            curr_TRACK_Center = get_center_of_box(TRACK_boxes[TRACK_box_i])

            curr_distance = math.dist(curr_TRACK_Center, new_OBJ_Center)

            if curr_distance < distance:
                distance = curr_distance
                trackerIndex = TRACK_box_i

        b_box = new_OBJ_boxes[OBJ_box_i]
        TRACK_expanded_boxes[trackerIndex].append((int(b_box[0]),int(b_box[1]),int(b_box[2]),int(b_box[3])))

    return TRACK_expanded_boxes

def get_center_of_box(boundingBox):

    boundingBox_Xcenter = int((boundingBox[0] + (boundingBox[0] + boundingBox[2])) / 2)
    boundingBox_Ycenter = int((boundingBox[1] + (boundingBox[1] + boundingBox[3])) / 2)

    return(boundingBox_Xcenter, boundingBox_Ycenter)

## Estimate Human Poses

In [115]:
def seperate_person(person_BB, image, finalImage):

    POSE_IMG = np.zeros(finalImage.shape, dtype=np.uint8)

    for curr_BB in person_BB:

        tl = curr_BB[0]
        tr = curr_BB[0]+curr_BB[2]
        bl = curr_BB[1]
        br = curr_BB[1]+curr_BB[3]

        #Seperate each product/contour into a new image by cropping the image to the bounding box of the product
        POSE_IMG[bl:br, tl:tr] = image[bl:br, tl:tr]

    return POSE_IMG

def get_human_pose_from_img(POSE_img_box, FINAL_img):

    IMG_HEIGHT = POSE_img_box.shape[0]
    IMG_WIDTH = POSE_img_box.shape[1]

    #Resize for prediction
    BLOB_HEIGHT=368
    BLOB_WIDTH=int((BLOB_HEIGHT/IMG_HEIGHT)*IMG_WIDTH)

    # Use the given image as input, which needs to be blob(s).
    imgBlob = cv2.dnn.blobFromImage(POSE_img_box, 1.0/255, (BLOB_WIDTH, BLOB_HEIGHT), (0,0,0), swapRB=True, crop=False)
    POSE_net.setInput(imgBlob)

    # Runs a forward pass to compute the POSE_MODEL output
    out = POSE_net.forward()
    # MobilePOSE_MODEL output [1, 57, -1, -1], we only need the first 19 elements
    out = out[:, :19, :, :]

    assert(len(POSE_body_parts) == out.shape[1])

    points = []
    points_conf = []
    points_low = []
    points_low_conf = []

    for i in range(len(POSE_body_parts)):
        # Slice heatmap of corresponding body's part.
        heatMap = out[0, i, :, :]

        # Originally, we try to find all the local maximums. To simplify a sample
        # we just find a global one. However only a single pose at the same time
        # could be detected this way.
        _, conf, _, point = cv2.minMaxLoc(heatMap)
        x = (IMG_WIDTH * point[0]) / out.shape[3]
        y = (IMG_HEIGHT * point[1]) / out.shape[2]

        # Add a point if it's confidence is higher than THRESHOLD.
        points.append((int(x), int(y)) if conf > POSE_threshold else (0,0))
        points_conf.append(conf if conf > POSE_threshold else 0.0)

        # A low confidence variant
        points_low.append((int(x), int(y)))
        points_low_conf.append(conf)

    for pair in POSE_part_pairs:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in POSE_body_parts)
        assert(partTo in POSE_body_parts)

        idFrom = POSE_body_parts[partFrom]
        idTo = POSE_body_parts[partTo]

        if not points[idFrom] == (0,0) and not points[idTo] == (0,0):
            cv2.putText(FINAL_img,partFrom,points[idFrom], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)
            cv2.putText(FINAL_img,partTo,points[idTo],cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)
            cv2.line(FINAL_img, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv2.ellipse(FINAL_img, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)
            cv2.ellipse(FINAL_img, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv2.FILLED)

    return points, points_conf, points_low, points_low_conf

## Labelling

In [116]:
def create_labelling_window(list_of_persons):

    global window
    global check_list
    global check_var_list

    window = tk.Tk()
    window.title("Labelling")
    window.rowconfigure(0, minsize=100, weight=1)
    window.columnconfigure(1, minsize=70, weight=1)
    window.resizable(False, False)

    fr_checks = tk.Frame(window)
    fr_btn = tk.Frame(window)

    check_list = []
    check_var_list = []

    for x in range(len(list_of_persons)):

        check_var_list.append(tk.IntVar())

        check_list.append(tk.Checkbutton(fr_checks, text = "Person " + str(x), variable = check_var_list[x], onvalue = 1, offvalue = 0, height = 2, width = 10))

        check_list[x].grid(row=x , column=0 , sticky="ew" , padx=5 , pady=5)

    btn_ok = tk.Button(fr_btn, text="OK", command=window.destroy)

    fr_checks.grid(row=0, column=0, sticky="ns")
    fr_btn.grid(row=1, column=0, sticky="ns")
    btn_ok.grid(row=0, column=0, sticky="ew", padx=5, pady=5)

## Prepare Data for export

In [117]:
def format_data(coord_loc_list, frame_shape):

    global check_var_list

    csv_list = []

    for x in range(len(coord_loc_list)):
        for y in range(0, len(coord_loc_list[x]),3):
            try:
                list_output_keypoints = []
                for z in range(len(coord_loc_list[x][y])):
                    for frame_count in range(3):
                        for coord_count in range(2):
                            list_output_keypoints.append( (coord_loc_list[x][y + frame_count][z][coord_count]) / frame_shape [coord_count])

                csv_list.append([("violent" if check_var_list[x].get() == 1 else "non-violent")] + list_output_keypoints)
            except:
                break


    return csv_list

def format_conf_data(conf_list):

    global check_var_list

    csv_list = []

    for person_i in range(len(conf_list)):
        for frame_i in range(0, len(conf_list[person_i]),3):
            try:
                list_output_keypoints = []
                for joint_i in range(len(conf_list[person_i][frame_i])):
                    for frame_count in range(3):
                        list_output_keypoints.append(conf_list[person_i][frame_i + frame_count][joint_i])

                csv_list.append([("violent" if check_var_list[person_i].get() == 1 else "non-violent")] + list_output_keypoints)
            except:
                break

    return csv_list


def cull_data(coord_loc_list, conf_list):
    removed_dub_list = []
    removed_dub_list_conf = []

    for x in range(len(coord_loc_list)):
        if coord_loc_list[x] not in removed_dub_list:
            removed_dub_list.append(coord_loc_list[x])
            removed_dub_list_conf.append(conf_list[x])

    return removed_dub_list, removed_dub_list_conf

## Export to CSV

In [118]:
def export_to_csv(keypoints_list, csv_col, export_loc):

    np_array_rows = np.array(keypoints_list)
    np_array_columns = np.array(csv_col)

    df = pd.DataFrame(np_array_rows)

    if not os.path.isfile(export_loc):
       df.to_csv(export_loc, index=False, header=np_array_columns)
    else: # else it exists so append without writing the header
       df.to_csv(export_loc, index=False, mode='a', header=False)

## Main Pipeline

In [119]:
def num_sort(received_string):
    return list(map(int, re.findall(r'\d+', received_string)))[0]

def DataExtraction(VIDEO_LOC):

    global window
    global check_list
    global check_var_list

    frameCount = 0
    OBJ_boxes = []

    poseLoc = []
    poseLoc_conf = []
    poseLoc_low = []
    poseLoc_low_conf = []

    video_reso = 0

    #Initialize the video stream
    MEDIA_RAW = cv2.VideoCapture(VIDEO_LOC)

    video_reso = (MEDIA_RAW.get(3),MEDIA_RAW.get(4))

    #Loop over the frames from the video stream
    while True:

        k = cv2.waitKey(1) & 0xFF

        if k == 27:
            MEDIA_RAW.release()
            cv2.destroyAllWindows()
            return True
            break

        #Grab the frame from the threaded video stream
        hasFrame, image = MEDIA_RAW.read()

        if not hasFrame:
            break

        finalImage = image.copy()
        frameCount += 1

        start = time.time()

        #Only Detect and Track People in the first frame
        if frameCount == 1:
            #Detect People in Image and return their Boxes Position
            OBJ_boxes = detect_individuals_from_image(image)

            #Pass Bounding boxes to Tracker to start tracking
            start_tracking_from_boxes(OBJ_boxes, image)

            #Prepare Pose Save Locations
            poseLoc = [[] for i in range(len(OBJ_boxes))]
            poseLoc_conf = [[] for i in range(len(OBJ_boxes))]
            poseLoc_low = [[] for i in range(len(OBJ_boxes))]
            poseLoc_low_conf = [[] for i in range(len(OBJ_boxes))]

            #Create Tkinter window
            create_labelling_window(OBJ_boxes)

        #Sync trackers with new frame
        track_using_trackers(image, finalImage)

        #Expand tracker bounding box with more accurate bounding boxes
        POSE_EST_LOC = expand_tracking_box(image, finalImage)

        #Apply Pose Estimation in Bounding Boxes of every person
        for person_BBs_i in range(len(POSE_EST_LOC)):

            #Create Image focusing on a single person
            POSE_IMG = seperate_person(POSE_EST_LOC[person_BBs_i], image, finalImage)

            pose_calc, pose_calc_conf, pose_calc_low, pose_calc_low_conf = get_human_pose_from_img(POSE_IMG, finalImage)

            #Estimate Pose on a single person
            poseLoc[person_BBs_i].append(pose_calc)
            poseLoc_conf[person_BBs_i].append(pose_calc_conf)
            poseLoc_low[person_BBs_i].append(pose_calc_low)
            poseLoc_low_conf[person_BBs_i].append(pose_calc_low_conf)

        #Resize for better output
        finalImage = cv2.resize(finalImage, (int((700/finalImage.shape[0])*finalImage.shape[1]),700))

        #Time taken
        cv2.putText(finalImage, f'Time: {time.time() - start}', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

        #show the output frame
        cv2.imshow(VIDEO_LOC, finalImage)

        #Tkinter window running
        window.mainloop()

    #Stop any videos
    MEDIA_RAW.release()

    #Close all windows
    cv2.destroyAllWindows()

    #Format the data for export
    formatted_data = format_data(poseLoc, video_reso)
    formatted_data_conf = format_conf_data(poseLoc_conf)
    formatted_data_low = format_data(poseLoc_low, video_reso)
    formatted_data_low_conf = format_conf_data(poseLoc_low_conf)

    #Remove any duplicate data
    finalised_data, formatted_data_conf = cull_data(formatted_data, formatted_data_conf)
    finalised_data_low, formatted_data_low_conf = cull_data(formatted_data_low, formatted_data_low_conf)

    #Export to CSV
    export_to_csv(finalised_data, DATASET_COL, DATASET_EXP_LOC)
    export_to_csv(formatted_data_conf, DATASET_COL_CONF, DATASET_EXP_LOC_CONF)
    export_to_csv(finalised_data_low, DATASET_COL, DATASET_EXP_LOC_LOW)
    export_to_csv(formatted_data_low_conf, DATASET_COL_CONF, DATASET_EXP_LOC_LOW_CONF)

    return False

## Main Method

In [120]:
end_process = False

#Go through each video in the dataset
for dataset_class in DATASET_CLASSES:
    for camera in DATASET_CAMS:

        CURRENT_DIR = DATASET_VIDEOS + dataset_class + camera

        list_video_names = listdir(CURRENT_DIR)
        list_video_names.sort(key=num_sort)

        #For each video run the pipeline
        for video_name in list_video_names:

            CURRENT_VID = CURRENT_DIR + video_name

            end_process = DataExtraction(CURRENT_VID)

            if end_process:
                break
            else:
                #Move finished video to another location
                shutil.move(CURRENT_VID, "Dataset/videos_processed/" + dataset_class + camera)

        if end_process:
            break
    if end_process:
        break

True